In [1]:
MIN_COUNT = 5
STRIDE = 15
LENGTH = 40

In [2]:
import pandas as pd

In [3]:
!pip install mwparserfromhell

   ---------------------------------------- 0.0/156.4 kB ? eta -:--:--
   --------------- ------------------------ 61.4/156.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 156.4/156.4 kB 1.6 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

load_dataset("olm/wikipedia", language="en", date="20220920")

FileNotFoundError: https://dumps.wikimedia.org/enwiki/20220920/dumpstatus.json

In [3]:
# Download EN wikipedia locally.

# import requests
# import io

# WIKIPEDIA = ["https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/0.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/1.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/2.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/3.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/4.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/5.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/6.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/7.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/8.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/9.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/10.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/11.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/12.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/13.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/14.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/15.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/16.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/17.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/18.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/19.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/20.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/21.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/22.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/23.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/24.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/25.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/26.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/27.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/28.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/29.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/30.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/31.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/32.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/33.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/34.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/35.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/36.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/37.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/38.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/39.parquet","https://huggingface.co/api/datasets/wikimedia/wikipedia/parquet/20231101.en/train/40.parquet"]

# for i, url in enumerate(WIKIPEDIA):
#     print(url)
#     pd.read_parquet(io.BytesIO(requests.get(url).content)).rename(columns={'id': 'article_id'}).to_parquet(f"wiki/wiki_{i}.parquet")

In [4]:
import pandas as pd

def select_top_counts(min_count):
    selected = {}
    with open('data/links.txt', 'r', encoding='utf-8', errors='replace') as file:
        for line in file:
            try:
                # Filter out hashtag pages
                if line[0] == '#':
                    continue

                # Filter out category pages
                skip = False
                for substring in ['Category:', 'File:', 'Wikipedia:', 'user:']:
                    if substring in line:
                        skip = True
                if skip:
                    continue
                    
                # Filter out aliases and sublinks
                for substring in ['#', '|']:
                    if substring in line:
                        line = line.split(substring, 1)[0]

                line = line.strip()

                if line in selected:
                    selected[line] += 1
                else:
                    selected[line] = 1
            except Exception as e:
                print(e)

    selected = pd.DataFrame(
        list(
            filter(
                lambda x: x['count'] >= min_count,
                sorted(
                    [
                        {
                            'lower_title': x.lower().strip(), 
                            'count': y
                        } 
                        for x, y in selected.items()
                    ],
                    key=lambda x: x['count'],
                    reverse=True
                )   
            )
        )
    )

    return selected

selected = select_top_counts(MIN_COUNT)
selected.__len__()

701412

In [5]:
import os
import pandas as pd
from typing import Set 

def filter_df(df, selected):
    df = df.rename(columns={'id': 'article_id'})
    df['lower_title'] = df.title.apply(lambda x: x.strip().lower())
    return df.merge(selected, on='lower_title')

In [6]:
first = True
for file in os.listdir('wiki'):
    print(file)
    if first:
        df = filter_df(pd.read_parquet(f'wiki/{file}'), selected)
        first = False
        continue
    df = pd.concat([df, filter_df(pd.read_parquet(f'wiki/{file}'), selected)], axis=0)
df = df.drop_duplicates(['article_id'])
print(len(df))

wiki_0.parquet
wiki_1.parquet
wiki_10.parquet
wiki_11.parquet
wiki_12.parquet
wiki_13.parquet
wiki_14.parquet
wiki_15.parquet
wiki_16.parquet
wiki_17.parquet
wiki_18.parquet
wiki_19.parquet
wiki_2.parquet
wiki_20.parquet
wiki_21.parquet
wiki_22.parquet
wiki_23.parquet
wiki_24.parquet
wiki_25.parquet
wiki_26.parquet
wiki_27.parquet
wiki_28.parquet
wiki_29.parquet
wiki_3.parquet
wiki_30.parquet
wiki_31.parquet
wiki_32.parquet
wiki_33.parquet
wiki_34.parquet
wiki_35.parquet
wiki_36.parquet
wiki_37.parquet
wiki_38.parquet
wiki_39.parquet
wiki_4.parquet
wiki_40.parquet
wiki_5.parquet
wiki_6.parquet
wiki_7.parquet
wiki_8.parquet
wiki_9.parquet
501716


In [11]:
m = selected.merge(df[['article_id', 'lower_title']], on='lower_title', how='left')
m = m[m.article_id.isna()]
m.to_parquet('data/selected_missing.parquet')
del(m)

In [7]:
df.drop('text', axis=1).to_parquet(f'data/articles_top{MIN_COUNT}_len{LENGTH}_stride{STRIDE}.parquet')

In [8]:
import json

buffer = [] 
BUFFER_SIZE = 100000
chunk_id = 0
articles_processed = 0

def clear_buffer(buffer):
    with open(f'data/chunks_top{MIN_COUNT}_len{LENGTH}_stride{STRIDE}.jsonl', 'a', encoding='utf-8') as file:
        for row in buffer:
            file.write(json.dumps(row) + '\n')
    return []

for _, row in df.iterrows():
    if articles_processed % 10000 == 0:
        print('processed article:', articles_processed)
    start = 0
    end = LENGTH
    text = row['text'].split()
    while end < len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start : end])
        })
        chunk_id += 1
        
        start += STRIDE 
        end += STRIDE

        if len(buffer) >= BUFFER_SIZE:
            buffer = clear_buffer(buffer)
    
    if start <= len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start :])
        })
        chunk_id += 1

    articles_processed += 1

if buffer:
    buffer = clear_buffer(buffer)

processed article: 0
processed article: 10000
processed article: 20000
processed article: 30000
processed article: 40000
processed article: 50000
processed article: 60000
processed article: 70000
processed article: 80000
processed article: 90000
processed article: 100000
processed article: 110000
processed article: 120000
processed article: 130000
processed article: 140000
processed article: 150000
processed article: 160000
processed article: 170000
processed article: 180000
processed article: 190000
processed article: 200000
processed article: 210000
processed article: 220000
processed article: 230000
processed article: 240000
processed article: 250000
processed article: 260000
processed article: 270000
processed article: 280000
processed article: 290000
processed article: 300000
processed article: 310000
processed article: 320000
processed article: 330000
processed article: 340000
processed article: 350000
processed article: 360000
processed article: 370000
processed article: 380000


In [9]:
# def print_text(text):
#     start = 0
#     end = 75
#     while end < len(text):
#         print(text[start : end])
#         start += 75
#         end += 75
#     print(text[end :])

# print_text(chunks.sample().iloc[0]['chunk'])